In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
import os
import wfdb
from scipy.signal import medfilt
from sklearn.preprocessing import OneHotEncoder

In [2]:
print(os.getcwd())

/Users/vegy-math808y/Speciale/masters


# preprocessing the AFDB dataset for deep learning algorithm

In [3]:
#patients = ['04043']

patients = ['04015','04043','04048','04126','04746','04908',
            '04936','05091','05121','05261','06426','06453','06995','07162',
            '07859','07879','07910','08215','08219','08378','08405','08434',
            '08455']
# id '00735' and '03665' is not included since no raw ECG values (.dat file) exists 

In [14]:
df = pd.DataFrame()

for patient_id in patients:
    file_name = os.getcwd() + '/data/afdb/' + patient_id
    annotation = wfdb.rdann(file_name, 'qrs')
    types = annotation.symbol
    
    values, counts = np.unique(types, return_counts=True)
    df1 = pd.DataFrame({'types':values, 'val':counts, 'patient_id':[patient_id]*len(counts)})
    df = pd.concat([df, df1],axis = 0)

Now, we can print the number of heartbeats for each heartbeat type as shown below:

In [15]:
df.groupby('types').val.sum().sort_values(ascending = False)

types
N    1128561
Name: val, dtype: int64

However, some of these symbols represent the "non-beat" beats and we need to filter them out!

For more information about the beat types, please see the links below: 

https://archive.physionet.org/physiobank/database/html/mitdbdir/tables.htm#testbeats


https://archive.physionet.org/physiobank/database/html/mitdbdir/intro.htm#symbols

In [4]:
non_beat = ['[','!',']','x','(',')','p','t','u','`','\'','^','|','~','+','s','T','*','D','=','"','@','Q','?'
            ,'L','R','V','/','f','F','j','a','E','J','e','S']
#abnormal_beat = ['L','R','V','/','A','f','F','j','a','E','J','e','S']

afib_beat = ['A']


Create a new column named ``class`` and initilaize it with -1. Then, change the values to 0 if is is ``N`` (normal beat) and to 1 if it is ``afib``. 


In [17]:
df['class'] = -1

### BEGIN SOLUTION
df.loc[df.types == 'N','class'] = 0
df.loc[df.types.isin(abnormal_beat), 'class'] = 1
### END SOLUTION

NameError: name 'abnormal_beat' is not defined


Load the ECG signal and annotation file for a single patient. The ``read_ecg`` function gets a path to the downloaded dataset and returns the signal values/amplitudes, annotation/label type, and the location of the annotations/labels.

sampling frequency of the AFDB ECG records is 250 Hz.




In [11]:
def read_ecg(path):
    
    # Read ECG signal
    record = wfdb.rdrecord(path)
    
    # Get the ECG signal
    ecg_sig = record.p_signal
    # Since there are two leads of ECGs for each record, we only select the first lead to work with.
    ecg_sig = ecg_sig[:,0]
    
    # Read corresponding annotation file
    #annot = wfdb.rdann(path, 'atr')
    
    ann = wfdb.rdann(path, 'atr')
    QRS = wfdb.rdann(path, 'qrs')
   
    #get RPeak location (index) 
    annot_sample = pd.Series(QRS.sample)
    
    #extract and assign labels to individual beats. 
    #Needed since beat types are only defined in intervals in .atr file
    Symb = pd.Series(ann.symbol)
    Samp = pd.Series(ann.sample)
    Rhythm = pd.Series(ann.aux_note)    

    df1 = pd.DataFrame({'Rpeak': QRS.sample})
    df1["label"]=np.nan
    ##print(df1.head(10))
    df2 = pd.DataFrame({'Rpeak': ann.sample,"label": ann.aux_note})
    ##print("dataframe2")
    ##print(df2.head(10))
    newdf = pd.concat([df1,df2], keys = ['1', '2'])
    newdf=newdf.sort_index(ascending=False) 
    #make sure that values from annotation file is on top even if Rpeaks are identical. Important for ffill
    newdf=newdf.sort_values(by=['Rpeak'])
    #perform forward ffill to get annotation to all Rpeaks in QRS.sample
    newdf=newdf.ffill()
    #discard all Rpeaks+Annotations from ann.sample. Avoid adding extra or duplicate beats 
    newdf=newdf.loc[['1']]
    
    #convert label col to series for further processing 
    annot_symbol = pd.Series(list(newdf["label"]))
    
    annot_symbol = annot_symbol.replace('(AFIB', 'A')
    annot_symbol = annot_symbol.replace('(N', 'N')
    
    return ecg_sig, annot_symbol, annot_sample

In [41]:
#path = os.getcwd() + '/data/afdb/' + patients[0]# This reads the first ECG records, which is record number ´00735´
path = os.getcwd() + '/data/afdb/' + '04043'
ecg_sig, annot_symbol, annot_sample = read_ecg(path)

annot_symbol.describe()
print(np.shape(ecg_sig))

df=pd.DataFrame({'types':[],'val':[], 'patient_id':[]})
values, counts = np.unique(annot_symbol, return_counts=True)
df1 = pd.DataFrame({'types':values, 'val':counts, 'patient_id':patients[0]*len(counts)})
df = pd.concat([df, df1],axis = 0)
df.groupby('types').val.sum().sort_values(ascending = False)


(9205760,)


,val
count,3.000000
mean,20638.333333
std,24181.208703
min,25.000000
25%,7329.500000
50%,14634.000000
75%,30945.000000
max,47256.000000


In [43]:


# Read ECG signal
record = wfdb.rdrecord(path)
    
# Get the ECG signal
ecg_sig = record.p_signal

### Filter using 2 consecutive moving median filters to obtain baseline which is the subtracted from raw signal. Used to remove baseline wander


In [22]:
from scipy.signal import medfilt

ecg_sig1 = ecg_sig[:1000]
sampfrq=250

#Define first window (200ms) - must be odd. therefore subtract 1
window200 = int(200*sampfrq/1000)-1 
#Define second window (600ms)
window600 = int(600*sampfrq/1000)-1
med1 = medfilt(ecg_sig1, kernel_size=window200)
med2 = medfilt(med1, kernel_size=window600)
corr = ecg_sig1-med2

##plot result
"""
# Create a Figure with 2 rows and 2 columns of subplots:
fig, ax = plt.subplots(4, 1)

# Index 4 Axes arrays in 4 subplots within 1 Figure: 
ax[0, 0].plot(ecg_sig1) #row=0, column=0
ax[0, 1].plot(med1) #row=1, column=0
ax[0, 2].plot(med2) #row=0, column=1
ax[0, 4].plot(corr) #row=1, column=1

plt.show()


"""

plt.subplot(2, 1, 1)
plt.title('Raw')
plt.plot(ecg_sig1)
plt.subplot(2, 1, 2)
plt.subplots_adjust(hspace=0.5)
plt.title('Corrected')
plt.plot(corr, color='red')
plt.show()


NameError: name 'ecg_sig' is not defined

In [5]:
#make filter as function to be called pr patient

def bw_filt(ecg_sig, sampfrq):
    #Define first window (200ms) - must be odd. therefore subtract 1
    window200 = int(200*sampfrq/1000)-1 
    #Define second window (600ms)
    window600 = int(600*sampfrq/1000)-1
    med1 = medfilt(ecg_sig, kernel_size=window200)
    med2 = medfilt(med1, kernel_size=window600)
    corr = ecg_sig-med2
    #return corrected signal
    return corr


In [6]:
#make lowpass filter function
#inspired by https://medium.com/analytics-vidhya/how-to-filter-noise-with-a-low-pass-filter-python-885223e5e9b7

from scipy.signal import butter,filtfilt

def butter_lowpass_filter(data, cutoff, sampfrq):
    # Get the filter coefficients 
    #order = 2       # sin wave can be approx represented as quadratic
    #order = 4 according to https://www.sciencedirect.com/science/article/pii/S0026269219306330
    b, a = butter(4, cutoff, btype='low', analog=False, fs=sampfrq)
    y = filtfilt(b, a, data)
    return y

Q2. Please complete the ``build_dataset`` function below to make the dataset by ignoring the **non-beats** heartbeats. You will use the ``read_ecg`` function to read each ECG reacords and then extract the single beats as described in the previous cell.

**Note:** 

1. The ``build_dataset`` function gets the list of patients (``patients``), the time interval (``interval``) before and after each heart peak ($\pm$3 seconds for example), sampling frequency (``fs``), which is 360 Hz for MIT-BIH dataset, and the list of abnormal beats symbols (``abnormal_beat``) as defined earlier.



2. The ``build_dataset`` function returns two matrices called ``X`` and ``Y``, which are extracted heartbeats and their coresponding labels (**normal** or **abnormal**), respectively.  It also returns the annotation symbol (``annot_symb``) for each individual extracted beat. It should be mentioned that matrix X rows and columns represent the number of beats and the values of beats (i.e. ``interval`` * ``fs``).

In [8]:
def build_dataset(patients, interval, fs, afib_beat):
    
    # Initialize the arrays
    num_cols = 2 * interval * fs  # This specify the length of the heartbeats to be extracted.
    #X = np.zeros((1,num_cols))
    #Y = np.zeros((1,1))
    X = np.ones((1,num_cols))
    Y = np.ones((1,1))
    annot_symb = []
    ids = []
    """
    df=pd.DataFrame({'types':[],'val':[], 'patient_id':[]})
    values, counts = np.unique(annot_symbol, return_counts=True)
    df1 = pd.DataFrame({'types':values, 'val':counts, 'patient_id':patients[0]*len(counts)})
    """
    # This list stores the number of extracted heartbeats for each patient.
    num_beats = []
    
    for patient_id in patients:
        file_path = os.getcwd() + '/data/afdb/' + patient_id
        
        ecg_sig, annot_type, annot_sample = read_ecg(file_path)
        
        # Since there are two leads of ECGs for each record, we only select the first lead to work with.
        ##moved further up in preprocess
        #ecg_sig = ecg_sig[:,0]
        
        
        #Filter baseline wander
        #############
        ecg_sig = bw_filt(ecg_sig, fs)
        #print("ecg_sig shape")
        #print(np.shape(ecg_sig))
        
        #Apply lowpass filter  at 100Hz
        ##**********
        ecg_sig = butter_lowpass_filter(ecg_sig, 100, fs)
        
        # We simply remove the "non-beats" beats from the df_annot dataframe and only keep "normal" and "abnormal" beats.
        df_annot = pd.DataFrame({'annot_type':annot_type,
                              'annot_sample':annot_sample})
        
        f.write(f'N before removal is {len(df_annot)}  for id {patient_id}\n')
        #print(df_annot.describe())
        
        df_annot = df_annot.loc[df_annot.annot_type.isin(afib_beat + ['N'])]
        f.write(f'N after removal is {len(df_annot)}  for id {patient_id}\n')
        
        #print(df_annot.describe())
        
        # The "make_XY" builds the x and y matrics for each extracted heartbeat
        x, y, symbol = make_XY(ecg_sig, df_annot, interval, num_cols, afib_beat)
        annot_symb = annot_symb + symbol
        num_beats.append(x.shape[0])
        X = np.append(X,x,axis = 0)
        Y = np.append(Y,y,axis = 0)
        id_toappend = []
        id_toappend = [patient_id] * len(x)
        ids = ids+id_toappend
        
    # remove top index from both X and Y since these were just used to initialize
    X = np.delete(X,0,axis=0)
    Y = np.delete(Y,0,axis=0)
    
    #oneHotEncode y
    print(Y[1:10])
    encoder = OneHotEncoder(handle_unknown='ignore')
    Y = encoder.fit_transform(Y).toarray()
    print(Y[1:10])

    #transform y to int
    print(Y.dtype)
    Y = np.rint(Y).astype(int)
    print(Y.dtype)
    print(Y[1:10])

    #confirm that arrays are same length
    print(np.shape(ids))   
    print(np.shape(X))
    print(np.shape(Y))
    
    return X, Y, annot_symb, ids


def make_XY(ecg_sig, df_annot, interval, num_cols, afib_beat):
    # this function builds the X,Y matrices for each beat
    # it also returns the original symbols for Y
    
    num_row = len(df_annot)
    print("new_row")
    print(num_row)

    x = np.zeros((num_row, num_cols))
    y = np.zeros((num_row,1))
    symbol = []
    
    # count the rows
    row_size = 0

    for annot_sample, annot_type in zip(df_annot.annot_sample.values, df_annot.annot_type.values):

        left = max([0,(annot_sample - interval*fs) ])
        right = min([len(ecg_sig),(annot_sample + interval*fs) ])
        xx = ecg_sig[left: right]
        if len(xx) == num_cols:
            x[row_size,:] = xx
            y[row_size,:] = int(annot_type in afib_beat)
            symbol.append(annot_type)
            row_size += 1
    x = x[:row_size,:]
    y = y[:row_size,:]
    return x, y, symbol


In [9]:
#patients =['04043','04048'] 
f = open("log.txt", "w")

interval =  3
fs = 250

###use to check with masters exctraction of beats/symbols
X, y, annot, ids = build_dataset(patients, interval, fs, afib_beat)

f.close()

new_row
44005
new_row
61890
new_row
39934
new_row
42860
new_row
47873
new_row
61129
new_row
47323
new_row
36793
new_row
49685
new_row
45534
new_row
54820
new_row
34837
new_row
55163
new_row
39298
new_row
60266
new_row
56587
new_row
35751
new_row
43279
new_row
59293
new_row
41971
new_row
58856
new_row
39850
new_row
59552


In [10]:
#np.save("data/afdb/dl6/X_BWcorr_LP_filt100.npy",np.array(X))
#np.save("data/afdb/dl6/y.npy",np.array(y))
#np.save("data/afdb/dl6/annot_data.npy",np.array(annot))


In [ ]:
#save dataset w/o lowpass filtering
np.save("data/afdb/dl6/X_BWcorr.npy",np.array(X))
#np.save("data/afdb/dl6/y.npy",np.array(y))
#np.save("data/afdb/dl6/annot_data.npy",np.array(annot))

In [34]:
#save dataset w/o filtering
np.save("data/afdb/dl6/X_raw.npy",np.array(X))
np.save("data/afdb/dl6/y.npy",np.array(y))
np.save("data/afdb/dl6/ids",np.array(ids))

In [29]:
print('before deleting first line')
print(X[0:4])
#X=np.delete(X,0,axis=0)
#print('efter deleting first line')
#print(X[0:4])




before deleting first line
[[ 0.17   0.19   0.16  ... -0.225 -0.205 -0.225]
 [ 0.16   0.14   0.095 ... -0.12  -0.125 -0.145]
 [ 0.18   0.16   0.1   ... -0.27  -0.25  -0.23 ]
 [ 0.29   0.325  0.33  ... -0.29  -0.25  -0.27 ]]


In [33]:
print(np.shape(X))
print(np.shape(y))
print(np.shape(annot))
print(np.shape(ids))

df = pd.DataFrame(data = y) 
values, counts = np.unique(y, return_counts=True)
df1 = pd.DataFrame({'types':values, 'val':counts})
    
df1
#'t':values, 'val':counts, 'patient_id':[patient_id]*len(counts)})
# df1 = pd.DataFrame({'types':values, 'val':counts, 'patient_id':[patient_id]*len(counts)})
#

(1116465, 1500)
(1116465, 1)
(1116465,)
(1116465,)


,types,val
0,0.0,608076
1,1.0,508389


In [ ]:

import random
random.seed(42)

try:
    del ids
    del X
    del y
except:
   pass

ids = np.load("data/afdb/dl6/ids.npy")
y = np.load("data/afdb/dl6/y.npy")
X = np.load("data/afdb/dl6/X_raw.npy")



interval =  3
fs = 250

#confirm that arrays are same length
print(np.shape(ids))   
print(np.shape(X))
print(np.shape(y))

df_id = pd.DataFrame({'id':ids})
df_x = pd.DataFrame(X)
df_y = pd.DataFrame(y)


#redundant only to use if not entire notebook is run from start.
patients = ['04015','04043','04048','04126','04746','04908',
            '04936','05091','05121','05261','06426','06453','06995','07162',
            '07859','07879','07910','08215','08219','08378','08405','08434',
            '08455']


#make test and train split (16/7)
pts_train = random.sample(patients, 16)
pts_test = [pt for pt in patients if pt not in pts_train]

print(pts_train)
print(pts_test)

X_train = df_x.loc[df_id.id.isin(pts_train)]
X_test = df_x.loc[df_id.id.isin(pts_test)]
y_train = df_y.loc[df_id.id.isin(pts_train)]
y_test = df_y.loc[df_id.id.isin(pts_test)]


"""

x_train = np.zeros((1,2*fs*interval))
print(np.shape(x_train))

toappend = []

#id_list = pd.DataFrame({'start': datacompress.index.values})
for pt in pts_train:
    left = start[pt]
    right = end[pt]
    #x_train = data_X[left:right]
    toappend = data_X[left:right]
    #x_train = np.concatenate((x_train, toappend), axis=0)
    x_train = np.append(x_train, toappend, axis=0)

print(np.shape(toappend))   
print("shape of dataset after loop is ")
print(np.shape(x_train))

"""


(1116465,)
(1116465, 1500)
(1116465, 1)
['08405', '04126', '04015', '05121', '05091', '08219', '04746', '08434', '06426', '06453', '08378', '04043', '05261', '04936', '08455', '06995']
['04048', '04908', '07162', '07859', '07879', '07910', '08215']


https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html


In [40]:
print(np.shape(data_train))

(780245, 1500)


In [12]:
#Generate filtered dataset in loop using different intervals

#patients =['04043','04048'] 
f = open("dataset_log.txt", "w")



fs = 250
intervals = [1,2,3,4,5]

for interval in intervals:
    f.write(f'*****dataset w/ interval length {interval*2} \n')
    ###use to check with masters exctraction of beats/symbols
    X, y, annot, ids = build_dataset(patients, interval, fs, afib_beat)

    path='data/afdb/dl'+str(interval*2)

    try:
        os.mkdir(path)
        print('made dir dl'+str(interval*2))
    except OSError as error:
        pass                

    np.save(path+'/X.npy',np.array(X))
    np.save(path+'/y.npy',np.array(y))
    np.save(path+'/ids',np.array(ids))
    np.save(path+'/annot_data.npy',np.array(annot))
    f.write('\n')
f.close()

new_row
44005


KeyboardInterrupt: 